In [0]:
current_version = spark.sql("DESCRIBE HISTORY db_workspace.my_schema.revenues_per_day").selectExpr("max(version)").collect()[0][0]

In [0]:
%sql
CREATE TABLE IF NOT EXISTS db_workspace.my_schema.films (
    id BIGINT GENERATED ALWAYS AS IDENTITY,
    title STRING,
    distributor STRING
)
USING DELTA;

In [0]:
df = (
    spark.read
        .option('readChangeData', 'true')
        .option("startingVersion", current_version)
        .table('db_workspace.my_schema.revenues_per_day')
        .select(
            'title',
            'distributor',
        )
        .dropDuplicates()
)

films = (
    df
        .join(
            spark.read.table('db_workspace.my_schema.films'), 
            on='title', 
            how='left_anti')
)

films.select('title').write.mode('overwrite').saveAsTable('db_workspace.my_schema.temp2')
films.write.mode('append').saveAsTable('db_workspace.my_schema.films')